Set initial parameter

In [2]:
import configparser
import json
import os
from os import path
from typing import List

from price_predict.utils import PandasUtils, TensorModelUtils
from price_predict.utils.PandasUtils import LabelType

config: configparser.ConfigParser = configparser.ConfigParser()
config.read("config.ini")

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
csv_file_path: str = config["DEFAULT"]["csv_file_path"]
train_data_file_path: str = "resources/train_data/trainData"
model_file_path: str = "resources/ml/pricePredict"

# Create needed resource folder
if not path.exists("resources"):
    os.mkdir("resources")
    os.mkdir("resources/ml")
    os.mkdir("resources/result")
    os.mkdir("resources/train_data")

label_version: str = config["TestModel"]["label_version"]

units_params: List[int] = json.loads(config["TestModel"]["units_params"])
hidden_layer_params: List[int] = json.loads(config["TestModel"]["hidden_layer_params"])
epochs: int = config["TestModel"].getint("epochs")
target_params: List[int] = json.loads(config["TestModel"]["target_params"])
use_small_dataset: bool = config["TestModel"].getboolean("use_small_dataset")
time_range_multiplier: int = config["TestModel"].getint("time_range_multiplier")
label_type: LabelType = LabelType[config["TestModel"]["label_type"]]

Build and test models

In [ ]:
import datetime
from matplotlib import pyplot as plt
import time

for hidden_layer in hidden_layer_params:
    for units in units_params:
        for target in target_params:
            run_number: int = len(os.listdir("resources/ml"))
            run_message: str = (
                    str(run_number) +
                    " - units: " + str(units) +
                    " - hidden layers: " + str(hidden_layer) +
                    " - epochs " + str(epochs) +
                    " - use small data: " + str(use_small_dataset) +
                    " - target: " + str(target) +
                    " - label type: " + str(label_type.name) +
                    " - label version: " + label_version
            )

            print("build module with run number: " + run_message)

            x_train, y_train, x_test, y_test = PandasUtils.get_labeled_data(
                csv_file_path=csv_file_path,
                use_small_dataset=use_small_dataset,
                time_range=target * time_range_multiplier,
                target=target,
                label_type=label_type
            )

            module_build_start_time: float = time.time()
            predictor = TensorModelUtils.build_model(
                x_train,
                y_train,
                units=units,
                hidden_layer=hidden_layer,
                epochs=epochs,
                run_number=run_number
            )
            module_build_end_time: float = time.time()

            print("start test prediction")
            test_predict = predictor.predict(x_test)
            # plot result
            plt.plot(x_test[1000 + target:3000 + target, x_test.shape[1] - 1, x_test.shape[2] - 1], color='black',
                     label='Real Price')
            plt.plot(y_test[1000 + target:3000 + target], color='red', label='target Price')
            plt.plot(test_predict[1000:3000], color='blue', label='Predicted Price')
            plt.title('Stock Price Prediction')
            plt.xlabel('Time')
            plt.ylabel('Stock Price')
            plt.legend()
            plt.savefig("resources/result/" + str(run_number) + ".png")
            plt.clf()

            result: int = 0
            for row in range(test_predict.shape[0]):
                if (test_predict[row] - test_predict[row - 1] > 0) == (y_test[row] - y_test[row - 1] > 0):
                    result += 1

            result_message: str = run_message + " - result " + str(result)
            build_time_sec: float = module_build_end_time - module_build_start_time
            result_message += " - module train time: " + str(datetime.timedelta(seconds=build_time_sec))
            result_file = open("resources/result/result.txt", "a")
            result_file.write(result_message + "\n")
            result_file.close()
            print("finish module with run number: " + result_message)